In [80]:
#dependencies
import pandas as pd
import math


In [81]:
#Bring in the dataset
gss = pd.io.stata.read_stata('GSS1996.dta', convert_categoricals=False)
gss.to_csv('gss_converted.csv')

In [82]:
#drop all instances where angry event was not recalled
gss=gss.dropna(subset=["chnang7"])

In [83]:
#Recode DV to dummy
gss["pray_anger"]=gss["chnang7"]
gss.loc[gss.chnang7 == 2, 'pray_anger'] = 0

In [84]:
#Recode sex to dummy, reference category = "male"
gss["sex_r"]=gss["sex"]
gss.loc[gss.sex == 1, 'sex_r'] = 0
gss.loc[gss.sex == 2, 'sex_r'] = 1

In [85]:
#Recode race to dummy, reference category = "white"
gss["race_r"]=gss["race"]
gss.loc[gss.race == 1, 'race_r'] = 0
gss.loc[gss.race > 1, 'race_r'] = 1

In [86]:
#Recode degree to dummy, reference category = "has degree" (note that in original study, ref cat was NO degree)
gss["degree_r"]=gss["degree"]
gss.loc[gss.degree <=3, 'degree_r'] = 0
gss.loc[gss.degree > 3, 'degree_r'] = 1

In [87]:
#Recode region to dummy, reference category = "From South"
gss["region_r"]=gss["region"]
gss.loc[(gss.region <=4) | (gss.region >7), 'region_r'] = 0
gss.loc[(gss.region >=5) & (gss.region <=7), 'region_r'] = 1

In [88]:
#Recode marital to new categorical variable with reference category = "single, never married" and divorced and separated categories combined (note that this differs slightly from the original study, which did not assign "0" to any value of the variable.)
#New categories:
#0=single, never married
#1=married
#2=divorced/separated
#3=widowed
gss["marital_r"]=gss["marital"]
gss.loc[(gss.marital ==5), 'marital_r'] = 0
gss.loc[(gss.marital ==4) | (gss.marital ==3), 'marital_r'] = 2
gss.loc[(gss.marital ==3), 'marital_r'] = 3

In [89]:
#Recode parent to dummy, reference category = "is parent"
gss["childs_r"]=gss["childs"]
gss.loc[(gss.childs >0), 'childs_r'] = 1


In [93]:
#Center the age variable
gss["age_c"]=gss["age"]-gss["age"].mean()